In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
except ImportError:
    pass

Mounted at /content/drive


In [ ]:
import pandas as pd

# Path to the file in Drive
file_path = '/content/drive/My Drive/capstone_project/data/raw/OnlineRetail.xlsx'

# Read it
df = pd.read_excel(file_path)

print(df.shape)
print(df.info())
df.describe()
# Preview
df.head()


(541909, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB
None


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [5]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [6]:
# Count missing values per column
missing_counts = df.isnull().sum()
missing_percent = (missing_counts / len(df)) * 100

# Unique values count (only for object columns)
unique_values = df.nunique()

# Sample values: show top 3 unique values for object columns
sample_values = {}
for col in df.select_dtypes(include=['object']).columns:
    sample_values[col] = df[col].dropna().unique()[:3]

# Data types
data_types = df.dtypes

# Print the Data Inventory Table
print("\n--- Data Inventory Report ---\n")
print(f"{'Column':<15}{'Data Type':<15}{'Non-Null Count':<15}{'Unique Values':<15}{'Sample Values'}")
for col in df.columns:
    dtype = str(data_types[col])
    non_null = df[col].notnull().sum()
    unique = unique_values[col] if col in unique_values else 'N/A'
    samples = sample_values[col] if col in sample_values else 'N/A'
    print(f"{col:<15}{dtype:<15}{non_null:<15}{unique:<15}{samples}")

# Missing values summary
print("\n--- Missing Values Summary ---\n")
print(missing_counts)
print("\nPercentage Missing:")
print(missing_percent.round(2))

# Quick check on negative quantity or unit price
print("\n--- Data Validity Checks ---\n")
print(f"Negative Quantity count: {(df['Quantity'] < 0).sum()}")
print(f"Zero or negative UnitPrice count: {(df['UnitPrice'] <= 0).sum()}")

# Check invoice numbers starting with 'C' (returns)
returns = df['InvoiceNo'].astype(str).str.startswith('C').sum()
print(f"Returns (InvoiceNo starting with 'C'): {returns}")



--- Data Inventory Report ---

Column         Data Type      Non-Null Count Unique Values  Sample Values
InvoiceNo      object         541909         25900          [536365 536366 536367]
StockCode      object         541909         4070           ['85123A' 71053 '84406B']
Description    object         540455         4223           ['WHITE HANGING HEART T-LIGHT HOLDER' 'WHITE METAL LANTERN'
 'CREAM CUPID HEARTS COAT HANGER']
Quantity       int64          541909         722            N/A
InvoiceDate    datetime64[ns] 541909         23260          N/A
UnitPrice      float64        541909         1630           N/A
CustomerID     float64        406829         4372           N/A
Country        object         541909         38             ['United Kingdom' 'France' 'Australia']

--- Missing Values Summary ---

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype

In [7]:
#Drop missing Description rows:*italicised text*
df = df[~df['Description'].isnull()]

In [8]:
#Drop returns (optional, if focusing on net sales):
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]

In [9]:
#filter invalid qty/price
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]

In [10]:
#drop duplicates
df = df.drop_duplicates()


In [11]:
#clean product names
df['Description'] = df['Description'].str.strip().str.upper()


In [12]:
#add date related columns
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])  # in case it's not already

df['InvoiceYear'] = df['InvoiceDate'].dt.year
df['InvoiceMonth'] = df['InvoiceDate'].dt.month
df['InvoiceDay'] = df['InvoiceDate'].dt.day
df['Weekday'] = df['InvoiceDate'].dt.day_name()
df['Hour'] = df['InvoiceDate'].dt.hour

In [13]:
def get_outliers_and_clean(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR

    # Identify outliers
    outliers = df[(df[column] < lower) | (df[column] > upper)]
    print(f"Number of outliers in '{column}': {outliers.shape[0]}")

    # Remove outliers
    df_cleaned = df[(df[column] >= lower) & (df[column] <= upper)]
    return df_cleaned

# Original shape before outlier removal
print("Original shape:", df.shape)

# Clean Quantity outliers
df = get_outliers_and_clean(df, 'Quantity')

# Clean UnitPrice outliers
df = get_outliers_and_clean(df, 'UnitPrice')

# Shape after cleaning
print("Cleaned shape:", df.shape)

Original shape: (524878, 14)
Number of outliers in 'Quantity': 27111
Number of outliers in 'UnitPrice': 37516
Cleaned shape: (460251, 14)


In [14]:
cleaning_log = {
    'original_shape': (541909, 8),
    'after_duplicates': df.drop_duplicates().shape,
    'outliers_removed': True,
    'columns_added': ['TotalPrice', 'Year', 'Month', 'Weekday', 'Hour'],
    'missing_values': df.isnull().sum().to_dict()
}
print("Data Cleaning Log:\n", cleaning_log)

Data Cleaning Log:
 {'original_shape': (541909, 8), 'after_duplicates': (460251, 14), 'outliers_removed': True, 'columns_added': ['TotalPrice', 'Year', 'Month', 'Weekday', 'Hour'], 'missing_values': {'InvoiceNo': 0, 'StockCode': 0, 'Description': 0, 'Quantity': 0, 'InvoiceDate': 0, 'UnitPrice': 0, 'CustomerID': 115551, 'Country': 0, 'TotalPrice': 0, 'InvoiceYear': 0, 'InvoiceMonth': 0, 'InvoiceDay': 0, 'Weekday': 0, 'Hour': 0}}


In [ ]:
df.to_csv('/content/drive/My Drive/capstone_project/data/cleaned/online_retail_cleaned.csv', index=False)